<a href="https://colab.research.google.com/github/Volt-github24/Groupe-4-info-l3-datasciences-tutoriels/blob/main/ngrams_tests_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
""""Obligatoire"""
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [22]:
""""Obligatoire"""
import os
os.chdir("/content/drive/")
%cd MyDrive/Collab_Stage_L3/
%ls

/content/drive/MyDrive/Collab_Stage_L3
'Copie de Rapport sur les NLG.gdoc'
 corpus.txt
'Generation de langage naturel.ipynb'
 Prise_en_main_Spacy.ipynb
"Rapport d'implementation du tutoriel sur les NLG.gdoc"
'Test Ngram corpus francais.ipynb'
 textgenrnn_weights.hdf5


Fonction de tokenisation pour tokeniser le texte d'entree


In [23]:
""""Obligatoire"""

# Fonction de Tokenisation

import spacy # importation de la librairie
!python -m spacy download fr_core_news_sm # pipeline pour le francais
nlp = spacy.load("fr_core_news_sm")

def return_token(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le texte de chaque token
    return [X.text for X in doc]    



2022-08-05 13:21:19.059343: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.3 MB 24.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


Tokenisation du texte  (corpus)

In [24]:
""""Obligatoire"""

# Corpus avec ponctuation
liste = []
with open("corpus.txt", "r", encoding='latin-1') as f:
  for line in f.readlines():
    liste.append(line)

# Tokenisation du corpus
corpus = [] # corpus est la liste des tokens
for line in liste:
  result = return_token(line)[:-1]
  corpus.append(result)

punc = [':', "!", "^", '.', '`', '~', ';', ',', '?', '...', '_']

Ici, je teste une nouvelle approche pour les ngrams, ainsi donc, le principe est que pour chaque sequence de mot, il faut savoir ses differents successeurs, cest a dire que quand j'ai une sequence de mots, je dois avoir un tableau qui me montre tous les successeurs possibles (a partir du corpus fournit ) pour cette sequence de mots, apres ce la, comme la machine ne comprend que les nombres, on va constituer un dictionnaire (mot : index) et un dictionnaire (index : mot) pour avoir l'index d'un mot et par la meme occasion le mot correspondant à un index

**UNIGRAMS**

In [25]:
""""Obligatoire"""

# UNIGRAMS

#Ici il s'agit du cas selon lequel la sequence de mot est un seul mot.
#construisons donc les dictionnaires en question  

def create_word_index_unigram(liste_tokens):
    word_to_index = {
    "START":0,
    "END":1
    }
    
    index_to_word = ['START', 'END']

    for speech in liste_tokens:
        for word in speech: # je parcours chaque phrase  je parcours ses tokens
            if word not in index_to_word: # et je n''ajoute un token que s'il n'est pas deja dans la liste des tokens 
                word_to_index[word] = len(index_to_word) # et dans le dictionnaire, la valeur pour la cle qui est le mot, est simplement a position de ce mot dans la liste des tokens
                #print(len(index_to_word))
                index_to_word.append(word) # ensuite j'ajoute ce token
                
    return word_to_index, index_to_word # et je retourne le dictionnaire et la liste

# dictionnaire, liste = create_word_index_unigram(liste_tokens)

Construisons à présent la matrice mots:successeurs

In [26]:
""""Obligatoire"""

import numpy as np

def create_unigram_matrix(word_to_index, index_to_word, texts):
    V = len(index_to_word) #taille de la matrice, la matrice est de v*v, v etant le nombre de mots differents qu'il ya dans le corpus'
    matrix = np.zeros((V, V)) # je cree la matrice en question

    for sentence in texts: #je parcours chaque phrase du corpus
        for i in range(len(sentence)):
            if i == 0:
                matrix[0, word_to_index[sentence[i]]] += 1 # Si le mot est le premier d'une phrase, je met la colonne de START à 1, pour marque que cest le debut d'une phrase
            else: 
                matrix[word_to_index[sentence[i-1]], word_to_index[sentence[i]]] += 1 # dans le cas ou il est un mot dans une phrase, j'incremenete la case mot successeur

        if i == (len(sentence)-1):
            matrix[word_to_index[sentence[i]], 1] += 1 # Si le mot est le dernier d'une phrase, je met la colonne de END à 1, pour marque que cest la fin d'une phrase

    return matrix

In [ ]:

#Get word to index and index to word for unigrams
word_to_index_unigram, index_to_word_unigram = create_word_index_unigram(corpus)

#Get unigram matrix
unigram_matrix = create_unigram_matrix(word_to_index_unigram,  index_to_word_unigram, corpus)

Pour la suite essayons de generer le texte(completer) avec cette matice construite sur la base de unigrams

Fonction pour retrouver le mot suivant :  il prend l'index du mot et retourne l'index du mot suivant

In [27]:
""""Obligatoire"""

import math
import random

def get_next_word(index_previous_ngram, matrix, show_nb_possibilities=False): # il prend l'index du mot et retourne l'index du mot suivant
    '''
    index_previous_ngram : int, represent the word index in the word_to_index matrix
    matrix : successor matrix, lines represent the n-gram, columns represent the successor

    1. Check how many successors are possible
    2. Select a random word from the possible words, to avoid repetition
    '''
    
    if (matrix[index_previous_ngram]>0).sum() < 2:
        nb_possible = 1
    elif (matrix[index_previous_ngram]>0).sum() > 9:
        nb_possible = 10
    else:
        nb_possible = (matrix[index_previous_ngram]>0).sum()

    if show_nb_possibilities==True:
        print('NUMBER POSSIBLE ', nb_possible)

    top_indexes = matrix[index_previous_ngram].argsort()[-nb_possible:] # constitue la liste des 10 mots suivants les plus succeptibles (avec le plus d'occurence) , 10 etant le nombre de possibiltes, il varie en trre 1 et 10
    random_index = math.floor(random.random()*nb_possible) # juste pour choisir aleatoirement  l'index du prochin mot parmi toutes les possibilites

    index_next_word = top_indexes[random_index] # on choisi un index aleatoirement dans cet ensemble des 10 possibilites
    
    if matrix[index_previous_ngram][index_next_word] == 0: # Si aucun mot n'est successeur, je renvoie le END
      index_next_word = 1 # C'est l'indice du END

    return index_next_word, matrix[index_previous_ngram][index_next_word]  # retourne l'index du prochain mot, et le nombre d'occurence de ce prochain mot sachant le mot precedent dans le corpus

Fonction pour compteter le texte avec unigram

In [ ]:
def get_sentence_unigram(previous_text, word_to_index, index_to_word,  matrix, nb_words_after=100, no_repetition=True):

    #1. tokenisation de la phrse en entree
    used_words = return_token(previous_text)

    #2. Recuperons le dernier mot de la demie phrase
    last_word = used_words[-1]

    print(previous_text, end=" ")
    index_last_word = word_to_index[last_word] # on recupere l'index du dernier mot de la demie phrase
    
    for i in range(nb_words_after):      #  cette boucle est la pour que si le break du bas ne s'execute pas, qu'on complete au plus le nombre de mots precisé
        index_last_word, occurence = get_next_word(index_last_word, matrix)
                
        if(index_last_word!=1):        # Je verifie que le mot que je veux ajouter n'est pas le mot END car cest la fin de la phrase, si cest le cas, je ne l'ajoute juste pas
          used_words.append(index_to_word[index_last_word])
          print(index_to_word[index_last_word], end = ' ') # j'affiche donc au fur et à mesure sans aller à la ligne
        else:
            break;

            

get_sentence_unigram("ENEO", word_to_index_unigram, index_to_word_unigram,unigram_matrix,nb_words_after=10)

AU final, avec les unigrams, le modele fait tellement d'erreurs car il ne tient pas compte du passe lointain de la phrase a completer, essayons donc les bigrams pour ameliorer les resultats
 Cest a dire on considere pour sequence de mots (deux mots)

**BIGRAMS**

Tout d'abord definissons d'abor les dictionnaires comme dans la methode unigram

In [ ]:
def create_word_index_bigram(texts):
    word_to_index = {
    "START START":0,
    "END":1
    }
    index_to_word = ['START START', 'END'] # exactement comme plus haut, sauf qu'ici les sequences sont constituees de deux mots, raison pour laquelle on a START START

    for speech in texts:
        for i in range(len(speech)):
            if i == 0:
                word = 'START ' + speech[0] # si je suis avec le premier mot d'une phrse pour stocker cette sequence, je concatene START avec ce premier mot
            else:
                word = speech[i-1] + ' ' + speech[i] # si cest dans la phrase, je le prend juste avec son precedent

            if word not in index_to_word:
                word_to_index[word] = len(index_to_word) # j'ajoute dans le dictionnaire 
                index_to_word.append(word) # et dans la liste , si biensur la sequence n'y etait pas encore
            
    return word_to_index, index_to_word

def create_bigram_matrix(word_to_index, index_to_word, word_to_index_unigram, texts): # elle prend le dictionnaire des bigrams, la liste des bigrams, le dictionnaire des unigram, car les mots suivant sont un sequence d'un seul mot, bien evidement et le texte corpus aussi
    V = len(index_to_word)
    V_unigram = len(word_to_index_unigram)
    matrix = np.zeros((V, V_unigram)) # on cree la matrice de taille , nombre de bigrams, nombre d'unigrams, les unigrams sont les successeurs

    for sentence in texts:
        for i in range(len(sentence)):
            if i == 0: # cas ou je suis avec le premier mot de la phrase
                matrix[word_to_index['START START'], word_to_index_unigram[sentence[0]]] += 1            # pour marquer le debut de la phrase
                if len(sentence)>1:
                  matrix[word_to_index['START '+sentence[0]], word_to_index_unigram[sentence[1]]] += 1      # pour marquer le debut de la phrase

            elif i >= (len(sentence)-1):
                matrix[word_to_index[sentence[i-1]+' '+sentence[i]], 1] += 1 # si je ne suis pas dans le debut de la phrase, mais au dernier mot plutot je met plutot la colonne de END  a 1 pour marquer la fin de la phrase.
       
            else: 
                matrix[word_to_index[sentence[i-1]+' '+sentence[i]], word_to_index_unigram[sentence[i+1]]] += 1 # si je suis au milieu de la phrase, eh bien j'uncrement simplement la case correspondante au successeur
    
    return matrix

In [ ]:
#Get word to index and index to word for bigrams
word_to_index_bigram, index_to_word_bigram = create_word_index_bigram(corpus)

#Get bigram matrix
bigram_matrix = create_bigram_matrix(word_to_index_bigram, index_to_word_bigram, word_to_index_unigram, corpus)

index_to_word_bigram[:20]

Generons maintenant le texte avec le modele bigram

In [ ]:
def get_sentence_bigram(previous_text, word_to_index, index_to_word, index_to_word_unigram, matrix, nb_words_after=20, no_repetition=True):
    
    #1. tokenisation de la phrse en entree
    used_words = return_token(previous_text)

    #2. Recuperons les deux dernier mot de la demie phrase
    if len(used_words)<1: 
      last_bigram = "START " + used_words[-1]
    else:
      last_bigram = used_words[-2] + " " + used_words[-1]  

    print(previous_text, end=" ")

    index_last_bigram = word_to_index[last_bigram]

    for i in range(nb_words_after):  
        
        index_next_word, occurence = get_next_word(index_last_bigram, matrix)
                
        if(index_next_word!=1):        # Je verifie que le mot que je veux ajouter n'est pas le mot END car cest la fin de la phrase, si cest le cas, je ne l'ajoute juste pas
          used_words.append(index_to_word_unigram[index_next_word])
          print(index_to_word_unigram[index_next_word], end = ' ') # j'affiche donc au fur et à mesure sans aller à la ligne
        else:
            break;

        index_last_bigram = word_to_index[used_words[len(used_words)-2] + ' ' + used_words[len(used_words)-1]] # juste pour mettre a jour la variable contenant l'id du dernier bigram, celle qu'on repasse a la fonction permettant d'avoir le successeur probable
            

In [ ]:
idx, occurence = get_next_word(word_to_index_bigram['suis pas'], bigram_matrix)
print(index_to_word_unigram[idx])
print(occurence)

In [ ]:
get_sentence_bigram("solde (espace)", 
                    word_to_index_bigram, 
                    index_to_word_bigram, 
                    index_to_word_unigram, 
                    bigram_matrix,
                   nb_words_after=100
                   )

L'efficacite des bigrams peut etre augmentee en passant aux trigrams, testons ce modele aussi
 Cest a dire on considere pour sequence de mots (trois mots)

**TRIGRAMS**

Tout d'abord definissons d'abor les dictionnaires comme dans la methode bigram

In [ ]:

def create_word_index_trigram(texts):
    word_to_index = {
    "START START START":0,
    "END":1
    }
    index_to_word = ['START START START', 'END']

    for speech in texts:
        for i in range(len(speech)):
            if i == 0:
                word = 'START START ' + speech[i]
            elif i==1:
                word = 'START ' + speech[i-1] + ' ' + speech[i]
            else:
                word = speech[i-2] + ' ' + speech[i-1] + ' ' + speech[i]

            #print(word)
            if word not in index_to_word:
                word_to_index[word] = len(index_to_word)
                index_to_word.append(word)
            
    return word_to_index, index_to_word

def create_trigram_matrix(word_to_index, index_to_word, word_to_index_unigram, texts):
    V = len(index_to_word)
    V_unigram = len(word_to_index_unigram)
    matrix = np.zeros((V, V_unigram))

    for sentence in texts:
        for i in range(len(sentence)):
            if i == 0:
                matrix[word_to_index['START START START'], word_to_index_unigram[sentence[0]]] += 1 
                if len(sentence)>1:                
                  matrix[word_to_index['START START '+sentence[0]], word_to_index_unigram[sentence[i+1]]] += 1 

            elif i ==1:
              if len(sentence)>2:
                matrix[word_to_index['START '+sentence[i-1] + ' ' +sentence[i]], word_to_index_unigram[sentence[i+1]]] += 1

            elif i >= (len(sentence)-1):
                matrix[word_to_index[sentence[i-2] +' '+ sentence[i-1]+' '+sentence[i]], 1] += 1
       
            else: 
                matrix[word_to_index[sentence[i-2]+' '+sentence[i-1]+' '+sentence[i]], word_to_index_unigram[sentence[i+1]]] += 1
    
    return matrix

In [ ]:
#Get word to index and index to word for trigrams
word_to_index_trigram, index_to_word_trigram = create_word_index_trigram(corpus)

#Get trigram matrix
trigram_matrix = create_trigram_matrix(word_to_index_trigram, index_to_word_trigram, word_to_index_unigram, corpus)

index_to_word_trigram[:20]

Generons maintenant le texte avec le modele trigram

In [ ]:
def get_sentence_trigram(previous_text, word_to_index, index_to_word, index_to_word_unigram, matrix, nb_words_after=20, no_repetition=True):

    #1. tokenisation de la phrse en entree
    used_words = return_token(previous_text)

    #2. Recuperons les trois derniers mot de la demie phrase
    if len(used_words)<2: 
      last_trigram = "START " + used_words[-1]
    elif len(used_words)<3:
      last_trigram = "START " + used_words[-2] + " " + used_words[-1]  
    else:
      last_trigram = used_words[-3] + " " + used_words[-2] + " " + used_words[-1] 
    
    print(previous_text, end=" ")

    index_last_trigram = word_to_index[last_trigram]

    for i in range(nb_words_after):  
        
        index_next_word, occurence = get_next_word(index_last_trigram, matrix)

        if(index_next_word!=1):        # Je verifie que le mot que je veux ajouter n'est pas le mot END car cest la fin de la phrase, si cest le cas, je ne l'ajoute juste pas
          used_words.append(index_to_word_unigram[index_next_word])
          print(index_to_word_unigram[index_next_word], end = ' ') # j'affiche donc au fur et à mesure sans aller à la ligne
        else:
            break;    
        
        index_last_trigram = word_to_index[used_words[len(used_words)-3] + ' ' + used_words[len(used_words)-2] + ' ' + used_words[len(used_words)-1]]
            

In [ ]:
get_sentence_trigram("Hors l'établissement d'une", 
                    word_to_index_trigram, 
                    index_to_word_trigram, 
                    index_to_word_unigram, 
                    trigram_matrix,
                   nb_words_after=20
                   )

Maintenant essayons d'ecrire un algorithme qui prendra en parametre la taille n des grams

**N GRAMS**

Creation des deux dictionnaire et liste

In [28]:
""""Obligatoire"""

def create_word_index_ngram(texts, ngram_size = 3): # La fonction prend dorenavant un parametre precisant la taille des grams (initialisee à 3)
    first_chaine = "" # Comme il faut creer une chaine de START de taille ngramsize, je l'initialise
    for _ in range(ngram_size):
      first_chaine = first_chaine + "START "
    first_chaine = first_chaine[:-1] # j'enleve le dernier caractere qui est l'espace, cest la chaine START START START...  qui a l'id 0
    word_to_index = {
    first_chaine:0,
    "END":1 # et END a l'id 1
    }
    index_to_word = [first_chaine, 'END'] # J'initialise la liste des mots

    for speech in texts: # je parcours chaque phrase de mon corpus
        for i in range(len(speech)): # pour chaque mot de la phrse, non pas mot mais token
            word = "" # variable qui va contenir la sequence des mots, la sequence a la taille de ngramsize
            if i < ngram_size -1 : # Dans cette condition je constitue la sequence, et ici specifiquement elle commence par START
                word = 'START ' * (ngram_size - i - 1)
                for k in range(i+1): # Je concatene donc le debut des start la avec les tokens suivants
                    word = word + speech[k] + ' '
            else: # Et ici je constitue les sequences qui ne commencent pas par START, genre ce sont les mots du milieu de la phrase, pas du debut
                for k in range(i-(ngram_size-1), i+1):
                    word = word + speech[k] + ' '

            word = word[:-1] # j'enleve l'espace de la fin
            if word not in index_to_word: # si la sequence n'est pas encore dans la liste je vais l'ajouter, a la derniere ligne
                word_to_index[word] = len(index_to_word) # et je l'ajoute aussi dans le dictionnaire avec son id est son emplacement  dans la liste des sequences
                index_to_word.append(word)
            
    return word_to_index, index_to_word # Je retourne le dictionnaire et la liste...

Creation de la matrice n-gram

In [29]:
""""Obligatoire"""

# Cette fonction aussi prend le ngramsize en parametre et cela doit etre le meme que celui qui a ete utilisé dans create_word_index_ngram
def create_ngram_matrix(word_to_index, index_to_word, word_to_index_unigram, texts, ngram_size = 3):
    V = len(index_to_word) 
    V_unigram = len(word_to_index_unigram)
    matrix = np.zeros((V, V_unigram)) # Je definis ma matrice, qui est de taille (nombre de sequence de mots * nombre de mots unigrams)

    first_chaine = "" # Je fabrique la meme premiere chaine la que je vais ajouter a la colonne 0 de la matrice
    for _ in range(ngram_size):
      first_chaine = first_chaine + "START "
    first_chaine = first_chaine[:-1]

    for sentence in texts:
        for i in range(len(sentence)): # Je parcours donc chaque token de la liste des tokens de la phrase en question
            if i < ngram_size-1: # Dans cette condition je verifie que le token actuel est parmi les premiers tokens de la phrase, cest a dire pour les cas ou il n'a pas tous les token precedent s pour avoir la sequence complete à ettre dans la matrice
                taille_start = ""
                if i == 0: # a chaque fois je vais mettre la chaine START START ... dans la premiere colonne de la matrice
                    matrix[word_to_index[first_chaine], word_to_index_unigram[sentence[i]]] += 1 # et mettre cette case à 1 pour marquer le debut d'une nouvelle phrase
                if len(sentence) > i+1: # si la phrse est asseez longue, je mets la suite des sequences (ici ce sont les sequences qui commencent par START )
                    # je fabrique la chaine qui commence par START à faire correspondre en ligne dans la matrice puis je concatene avec les premiers mots et je fais la correspondance dans la matrice cad chercher son successeur
                    taille_start = "START " * (ngram_size-i-1) 
                    for j in range(i+1):
                        taille_start = taille_start + sentence[j] + " "
                    matrix[word_to_index[taille_start[:-1]], word_to_index_unigram[sentence[i+1]]] += 1  # j'incremente donc la case correspondant a la sequence et au successeur

            elif i >= (len(sentence)-1): # Il s'agit ici du cas ou je suis au dernier caractere de la phrase
                # je fabrique la chaine dont je cherche a mettre le successeur à END et la taille de la chaine est en fonction de la taille du ngram
                taille_chaine = ""
                for l in range(i-(ngram_size-1), i+1):
                    taille_chaine = taille_chaine + sentence[l] + " "
                matrix[word_to_index[taille_chaine[:-1]], 1] += 1 # je met la colonne de END à 1
            else: # Il s'agit donc du cas où je suis au milieur de la phrase, quand je tombe sur un mot (ou token), je prends simplement ses precedents, autant qu'il en faut pour faire correspondre a la taille de ngramsize
                taille_chaine = "" 
                for l in range(i-(ngram_size-1), i+1): # Je concatene ce mot et ses precedents
                    taille_chaine = taille_chaine + sentence[l] + " "
                matrix[word_to_index[taille_chaine[:-1]], word_to_index_unigram[sentence[i+1]]] += 1 # Puis j'increment la case correspondant e avec le sucesseur en colonne 
    
    return matrix    # Je retourne ensuite la matrice, toute faite

Appelons maintenant ces fonctions pour constituer notre matrice (le noeud de la chose)

In [31]:
#Get word to index and index to word for trigrams
word_to_index_ngram, index_to_word_ngram = create_word_index_ngram(corpus, 2)

#Get ngram matrix
ngram_matrix = create_ngram_matrix(word_to_index_ngram, index_to_word_ngram, word_to_index_unigram, corpus, 2)

index_to_word_ngram[:20]

Comme la methode de detection du mot suivant est deja fonctionnelle, implementons la fonction permettant de generer la suite de la phrase

In [41]:
""""Obligatoire"""

def get_sentence_ngram(previous_text, nb_words_after=100):

    #1. tokenisation de la phrase en entree
    used_words = return_token(previous_text)
    print(len(used_words))

    #2. Recuperons les n derniers mot de la demie phrase si n<10 et n sinon
    
    last_ngram = ""

    if len(used_words) == 0: # S'il n'entre rien genre une chaine vide, je lui retourne ce message d'erreur
        return "Entrez un texte avec au moins un caractere."

    elif len(used_words) < 10: # Cas ou la demie phrase a plus de 10 caracteres, je considere ngram comme une sequence de len(used_words)
      ngram_size = len(used_words) # la taille des sequences à former est simplement la taille de la demie phrase entrée
      
      for h in range(ngram_size, 0, -1):
          last_ngram = last_ngram + used_words[-h] + " "

    else : # Cas ou la demie phrase a plus de 10 caracteres, je considere ngram comme une sequence de 10
      
      ngram_size = 10 # la taille des sequences à former est simplement 10
      for h in range(ngram_size, 0, -1):
          last_ngram = last_ngram + used_words[-h] + " "

    # 3. Fabriquons maintenant les dictionnaires, la liste des sequences et la matrice en fonction de la taille des ngrams, 

    #Get word to index and index to word for unigrams
    word_to_index_unigram, index_to_word_unigram = create_word_index_unigram(corpus)

    #Get word to index and index to word for ngrams
    word_to_index_ngram, index_to_word_ngram = create_word_index_ngram(corpus, ngram_size)

    #Get ngram matrix
    matrix = create_ngram_matrix(word_to_index_ngram, index_to_word_ngram, word_to_index_unigram, corpus, ngram_size)
    
    last_ngram = last_ngram[:-1] # Pour enlever l'espace de la fin

    result = previous_text + " "

    index_last_ngram = word_to_index_ngram[last_ngram]

    for i in range(nb_words_after):  
        
        index_next_word, occurence = get_next_word(index_last_ngram, matrix)

        if(index_next_word!=1):        # Je verifie que le mot que je veux ajouter n'est pas le mot END car cest la fin de la phrase, si cest le cas, je ne l'ajoute juste pas
          used_words.append(index_to_word_unigram[index_next_word])
          result = result + index_to_word_unigram[index_next_word] + " "
        else:
            break;    

        last_ngram = ""
        for h in range(ngram_size, 0, -1):
          last_ngram = last_ngram + used_words[-h] + " "    
        index_last_ngram = word_to_index_ngram[last_ngram[:-1]]
    return result[:-1]

In [40]:
get_sentence_ngram("Merci de consulter vos mails et messages spams,", 100)

9


"Merci de consulter vos mails et messages spams, nous vous avons fais parvenir la fiche de demande de service , bien vouloir l' imprimer , le remplir convenablement(en vous assurant de cocher uniquement le service C - Online , déblocage compte et aussi réinitialisation mot de passe ) . Accompagnez ce document de votre copie de passeport sur laquelle vous signez 03 fois . Tous vos documents sous le format PDF s' il vous plaît ."